# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [183]:
from openai import OpenAI
from dotenv import load_dotenv
import ollama
from anthropic import Anthropic 
from IPython.display import display, Markdown, update_display
import gradio as gr
import os

In [182]:
# MODEL_GPT = 'gpt-5-mini-2025-08-07'
MODEL_GPT="o4-mini-2025-04-16"
MODEL_CLAUDE = "claude-sonnet-4-20250514"
MODEL_OLLAMA = 'qwen3:8b'
load_dotenv(override=True)

True

In [180]:
system_message = "You are a tech wizard. Take questions from your apprentice and provide deep but easy to understand explanations for questions in markdown format suitable for Gradio that runs in browser. Be a good teacher"

In [181]:
openai = OpenAI()
claude = Anthropic()

In [179]:
import subprocess
import tempfile
import os

def mermaid_to_svg(uml_definition: str, output_path: str = None) -> str:
    """
    Convert a Mermaid UML definition to an SVG file.

    Args:
        uml_definition (str): Mermaid UML definition.
        output_path (str): Path where SVG will be saved. If None, a temp file is used.

    Returns:
        str: Path to the generated SVG file.
    """
    # Ensure mermaid-cli (mmdc) is installed
    if not shutil.which("mmdc"):
        raise RuntimeError("Mermaid CLI (mmdc) not found. Install with: npm install -g @mermaid-js/mermaid-cli")

    # Create a temporary .mmd file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mmd", mode="w") as tmp_file:
        tmp_file.write(uml_definition)
        tmp_file_path = tmp_file.name

    # If no output path provided, generate one
    if output_path is None:
        output_path = tmp_file_path.replace(".mmd", ".svg")

    # Run Mermaid CLI
    subprocess.run(
        ["powershell", "C:/Users/kwojn/AppData/Roaming/npm/mmdc.ps1", "-i", tmp_file_path, "-o", output_path],
        check=True
    )

    # Clean up .mmd file
    os.remove(tmp_file_path)

    return output_path


In [178]:
import json
def get_tools():
    return [{
        "type": "function",
        "name": "generate_diagram",
        "description": "Generates image of a diagram from Mermaid input",
        "parameters": {
            "type": "object",
            "properties": {
                "definition": {
                    "type": "string",
                    "description": "Mermaid diagram definition"
                }
            },
            "required": ["definition"],
            "additionalProperties": False
        },
        "strict": True
    }]

def generate_diagram(definition):
    return mermaid_to_svg(definition)

def get_call_function(tool):
    match tool.name:
        case "generate_diagram":
            return generate_diagram
def handle_tools(tool_calls):
    output = []
    for tool in tool_calls:
        print(tool)
        args = json.loads(tool.arguments)
        func = get_call_function(tool)
        result = func(**args)
        output.append({
            "type": "function_call_output",
            "call_id": tool.call_id,
            "output": json.dumps(result)
        })
    return output

In [199]:
def ask_ollama(system_prompt, prompt, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": prompt}]
    response = ollama.chat(
        model=MODEL_OLLAMA,
        messages=messages,
        tools=get_tools(),
        think=True
    )
    print(response)
    if "tool_calls" in response["message"]:
        handle_tools(response["message"]["tool_calls"])
    if "thinking" in response.message:
        yield gr.ChatMessage(
            role="assistant",
            content=response.message.thinking,
            metadata={"title": "Thought a bit", 
                      "status": "done",
                      "duration": response.total_duration / 1e9
                     },
        )
    yield gr.ChatMessage(role="assistant", content=response.message.content)

previous_openai_id = None
def ask_openai(system_prompt, prompt, history):
    global previous_openai_id
    inputs = [{"role": "user", "content": prompt}]
    response = openai.responses.create(
        model=MODEL_GPT,
        input=inputs,
        instructions=system_prompt,
        previous_response_id=previous_openai_id,
        tools=get_tools(),
        max_output_tokens=2000
    )
    previous_openai_id = response.id
    print(response)
    tool_calls = list(filter(lambda item: item.type == "function_call", response.output))
    if len(tool_calls) > 0:
        tool_responses = handle_tools(tool_calls)
        for tool in tool_responses:
            yield gr.ChatMessage(
                role="assistant",
                content=tool["output"],
                metadata={"title": "Generated diagram", 
                          "status": "done",
                         },
            )
        response = openai.responses.create(
            model=MODEL_GPT,
            input=inputs + tool_responses,
            instructions=system_prompt,
            previous_response_id=previous_openai_id,
            tools=get_tools(),
            max_output_tokens=2000
        )
    for message in response.output:
        print(message)
        if message.type == "reasoning" and message.status:
            yield gr.ChatMessage(
                role="assistant",
                content='\n'.join([thinking.text for thinking in message.content]),
                metadata={"title": "Thought a bit", 
                          "status": "done",
                         },
            )
        elif message.type == "message":
            for content in message.content:
                yield gr.ChatMessage(role="assistant", content=content.text)
    

def select_ai(provider):
    if provider == "OpenAI":
        return ask_openai
    elif provider == "Local":
        return ask_ollama
    raise "AI provider not supported"

def chat(prompt, history, provider):
    func = select_ai(provider)
    history = [{"content": hist["content"], "role": hist["role"]} for hist in history]
    responses = func(system_message, prompt, history)
    for response in responses:
        history.append(response)
    return history

In [200]:
def transcribe(audio_file):
    print(audio_file)
    if audio_file is None:
        return ""
    
    with open(audio_file, "rb") as f:
        transcript = openai.audio.transcriptions.create(
            model="whisper-1",
            file=f
        )
    print(transcript)
    return transcript.text

def handle_transcribe_btn(audio, history, model):
    text = transcribe(audio)
    if text.strip() == "":
        return history
    response = chat(text, history, model)
    print(response)
    return response
    

In [201]:
with gr.Blocks() as ui:
    provider = gr.Dropdown(["OpenAI", "Local"], label="Select AI", value="Local")
    with gr.Row():
        voice=gr.Microphone(type="filepath")
        transcribe_btn = gr.Button("Transcribe")
    chatter = gr.ChatInterface(fn=chat, type="messages", additional_inputs=[provider])
    transcribe_btn.click(
        handle_transcribe_btn,
        [voice, chatter.chatbot, provider],
        chatter.chatbot
    )
ui.launch()

* Running on local URL:  http://127.0.0.1:7925
* To create a public link, set `share=True` in `launch()`.


C:\Users\kwojn\AppData\Local\Temp\gradio\f4727ea0a9ce35cbabd9b14ebc36d20949c315d8facd7a39b0a3ecd52182c94b\audio.wav
Transcription(text='Halo, raz, dwa, trzy, cztery, pięć.', logprobs=None, usage=UsageDuration(duration=None, type='duration', seconds=2))
model='qwen3:8b' created_at='2025-08-19T18:59:41.1873424Z' done=True done_reason='stop' total_duration=20508081300 load_duration=5817695200 prompt_eval_count=175 prompt_eval_duration=687373700 eval_count=275 eval_duration=13993477500 message=Message(role='assistant', content='Hello! It looks like you\'re saying "Hello" in Polish ("Halo") and listing numbers from one to five in Polish:  \n1. **raz** (one)  \n2. **dwa** (two)  \n3. **trzy** (three)  \n4. **cztery** (four)  \n5. **pięć** (five)  \n\nAre you learning Polish, or is there something specific you\'d like to explore? 😊', thinking='Okay, the user sent "Halo, raz, dwa, trzy, cztery, pięć." Let me break this down. "Halo" is Polish for "Hello," so they might be greeting me. Then ther